# Get networks

## Get TDDC

Input file(s):
* `../data/csv/sayles_data_90pct.csv`

Output file(s):
* [Optional] `../data/pickle/h.p` & `../data/pickle/TDDC.p`
    * TDDC[trial][seg] = respectively, scalor product *h* & TDDC values for each pair in a given segment. Shape: (N, N, num_tau, num_frames)
    * The produced .p file is very large (36.26 GB). I didn't save it since it's not being used, but it can always be reproduced. To reproduce this, uncomment all the TDDC codes in the code below.
* `../data/pickle/index.p`
    * index[trial][seg] = tau indices for max TDDC values at every time t for each pair in a given segment. Shape: (N, N, num_frames)

In [ ]:
## uncomment some lines to save h & TDDC values.

import pandas as pd
import pickle
import os

import config
from utils.get_TDDC import get_TDDC

df = pd.read_csv('../data/csv/sayles_data_90pct.csv')
h, TDDC = {}, {}
index = {}

for trial in df['trial'].unique():
    print('trial', int(trial), end=' ')
    trial_df = df[(df['trial']==trial)]
    h[trial], TDDC[trial] = {}, {}
    index[trial] = {}

    print('-- segment', end=' ')

    for seg in trial_df['segment'].unique():
        print(seg, end=' ')
        seg_df = trial_df[trial_df['segment']==seg]

        ## get velocities for the segment (ignore the first frame with NaNs)
        seg_xvel = seg_df.pivot(index='frame_segment', columns='ID', values='x_vel').to_numpy()[1:] # (num_frames, N)
        seg_yvel = seg_df.pivot(index='frame_segment', columns='ID', values='y_vel').to_numpy()[1:]

        ## compute TDDC & max tau
        ## - h (scaler product) & TDDC : (N, N, num_tau, num_frames)
        ## - index (indices of max tau) : (N, N, num_frames)
        _, _, index[trial][seg] = get_TDDC(seg_xvel, seg_yvel, SAMP_FREQ=config.SAMP_FREQ)
        # h[trial][seg],TDDC[trial][seg], index[trial][seg] = get_TDDC(seg_xvel, seg_yvel, SAMP_FREQ=config.SAMP_FREQ)
    #     break
    # break
    print(end='\n')

if not os.path.exists('../data/pickle'):
    os.makedirs('../data/pickle')

print("saving...")
# pickle.dump(h, open('../data/pickle/sayles_h.p', 'wb'))
# pickle.dump(TDDC, open('../data/pickle/sayles_TDDC.p', 'wb'))
pickle.dump(index, open('../data/pickle/sayles_index.p', 'wb'))
print("saved!")

## Get network weights

Input file(s):
* `../data/csv/sayles_data_90pct.csv`
* `../data/pickle/index.p` : TDDC tau indices 

Output file(s):
* `../data/pickle/sayles_weights_unpruned_500ms.p` : network weights without any pruning.
    * A[trial][seg] : (num_networks, N, N)
* `../data/pickle/sayles_weights_pruned_500ms.p` : network weights after pruning is applied.
    * A[trial][seg] : (num_networks, N, N)

In [2]:
import pandas as pd
import pickle
import config
from utils.get_network import get_network_weights

# ===== CHANGE THIS ========================================
ntwk_window_size = 0.5  # config.NTWK_WINDOW_SIZE   # in s
# ==========================================================

## init
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')
index = pickle.load( open( "../data/pickle/index.p", "rb" ) )
# constant values
SAMP_FREQ = config.SAMP_FREQ
tau = config.TAU
ntwk_window_size_ms = int(ntwk_window_size * 1000)

print('computing network weights (window size:', ntwk_window_size, 's)')

## compute network weights
unpruned_weights, pruned_weights = {}, {}
for trial in index.keys():
    print('trial', int(trial), '-- segment', end=' ')
    unpruned_weights[trial], pruned_weights[trial] = {}, {}
    trial_df = df[(df['trial']==trial)]

    for seg in index[trial].keys():
        print(seg, end=' ')
        seg_df = trial_df[trial_df['segment']==seg]

        x = seg_df.pivot(index='frame_segment', columns='ID', 
                         values='x').to_numpy()
        y = seg_df.pivot(index='frame_segment', columns='ID', 
                         values='y').to_numpy()
        heading = seg_df.pivot(index='frame_segment', columns='ID', 
                               values='heading').to_numpy()
        
        seg_length = x.shape[0] / SAMP_FREQ     # length of the segment in seconds
        # disregard the segment if too short
        if (seg_length < ntwk_window_size/2):
            print('(X)', end=' ')
            continue
        
        # weights[trial][seg] : (num_networks, N, N)
        unpruned_weights[trial][seg] = get_network_weights(index[trial][seg], 
                                                           x, y, heading, SAMP_FREQ, 
                                                           pruning=False,
                                                           TAU=tau,
                                                           NTWK_WINDOW_SIZE=ntwk_window_size)    
        pruned_weights[trial][seg] = get_network_weights(index[trial][seg], 
                                                         x, y, heading, SAMP_FREQ, 
                                                         pruning=True,
                                                         TAU=tau,
                                                         NTWK_WINDOW_SIZE=ntwk_window_size)
    print(end='\n')


print("\nsaving...")
pickle.dump(unpruned_weights, 
            open(f'../data/pickle/sayles_weights_unpruned_{ntwk_window_size_ms}ms.p', 'wb'))
pickle.dump(pruned_weights, 
            open(f'../data/pickle/sayles_weights_pruned_{ntwk_window_size_ms}ms.p', 'wb'))
print("saved!")

computing network weights (window size: 4 s)
trial 1 -- segment 1 2 3 
trial 2 -- segment 1 2 3 4 5 
trial 3 -- segment 1 2 3 4 
trial 4 -- segment 1 2 3 4 5 
trial 5 -- segment 1 2 3 (X) 4 5 
trial 6 -- segment 1 2 3 4 5 
trial 7 -- segment 1 2 5 7 
trial 8 -- segment 1 2 3 4 5 
trial 9 -- segment 1 2 4 5 
trial 10 -- segment 1 3 4 
trial 11 -- segment 1 2 3 
trial 12 -- segment 1 2 4 

saving...
saved!


## Plot TDDC heat maps

In [ ]:
# plot for every pair, every segment
# NOTE: it will take forever, so I do not recommend running this.
# instead, run it for a specific trial/segment (see the next code block)

import pandas as pd

import config
from utils.get_TDDC import get_TDDC, plot_TDDC_ij

# used for the title
# from https://www.geeksforgeeks.org/how-to-print-superscript-and-subscript-in-python/
def get_sub(x):
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans(''.join(normal), ''.join(sub_s)) 
    return x.translate(res)

df = pd.read_csv('../data/csv/sayles_data_90pct.csv')

if not os.path.exists('../data/output'):
    os.makedirs('../data/output')

for trial in df['trial'].unique():
    print('trial', int(trial), end=' ')
    trial_df = df[(df['trial']==trial)]

    print('-- segment', end=' ')

    for seg in trial_df['segment'].unique():
        print(seg, end=' ')
        seg_df = trial_df[trial_df['segment']==seg]

        start_t = min(seg_df.frame_segment.unique() * config.SAMP_FREQ)
        end_t = max(seg_df.frame_segment.unique() * config.SAMP_FREQ)
        
        ## get velocities for the segment (ignore the first frame with NaNs)
        seg_xvel = seg_df.pivot(index='frame_segment', columns='ID', values='x_vel').to_numpy()[1:] # (num_frames, N)
        seg_yvel = seg_df.pivot(index='frame_segment', columns='ID', values='y_vel').to_numpy()[1:]

        ## compute TDDC & max tau
        ## - TDDC : (N, N, num_tau, num_frames)
        ## - index (indices of max tau) : (N, N, num_frames)
        _, TDDC, index = get_TDDC(seg_xvel, seg_yvel, SAMP_FREQ=config.SAMP_FREQ)
        N = TDDC.shape[0]

        for i in range(N):
            for j in range(N):
                if i != j:
                    # print(i+1,j+1)
                    title = 'Trial'+str(trial)+' Seg'+str(seg)
                    title += ': TDDC{}(t,$\\tau$)'.format(get_sub(str(i+1)+'-'+str(j+1)))
                    for form in ['png','svg']:
                        plot_TDDC_ij(TDDC[i,j,:,:], index[i,j,:],
                                    SAMP_FREQ=config.SAMP_FREQ,
                                    title=title,
                                    saved=True,
                                    output_folder=f'../output/sayles/{form}/TDDC_heatmaps/',
                                    output_file=f'TDDC_trial{int(trial)}_seg{seg}_i{i+1}_j{j}',
                                    output_format=form)

In [ ]:
# plot only for a given trial/segment

import pandas as pd
import config
from utils.get_TDDC import get_TDDC, plot_TDDC_ij

# ===== change here =====
trial = 1
seg = 1
# None to plot all, or a list of indices (ID - 1) to plot
i_plotted = [0,8]     # None
j_plotted = [8,0]     # None

saved = True        # whether SSSto save or display the figures
# =======================

print(f'plotting figures for trial {trial} seg {seg} (saved in in ../output/sayles/[form]/TDDC_heatmaps/)')

# used for the title
# from https://www.geeksforgeeks.org/how-to-print-superscript-and-subscript-in-python/
def get_sub(x): 
    normal = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+-=()"
    sub_s = "ₐ₈CDₑբGₕᵢⱼₖₗₘₙₒₚQᵣₛₜᵤᵥwₓᵧZₐ♭꜀ᑯₑբ₉ₕᵢⱼₖₗₘₙₒₚ૧ᵣₛₜᵤᵥwₓᵧ₂₀₁₂₃₄₅₆₇₈₉₊₋₌₍₎"
    res = x.maketrans(''.join(normal), ''.join(sub_s)) 
    return x.translate(res)

df = pd.read_csv('../data/csv/sayles_data_90pct.csv')
if not os.path.exists('../data/output'):
    os.makedirs('../data/output')
    
trial_df = df[(df['trial']==trial)]
seg_df = trial_df[trial_df['segment']==seg]

start_t = min(seg_df.frame_segment.unique() * config.SAMP_FREQ)
end_t = max(seg_df.frame_segment.unique() * config.SAMP_FREQ)

## get velocities for the segment (ignore the first frame with NaNs)
seg_xvel = seg_df.pivot(index='frame_segment', columns='ID', values='x_vel').to_numpy()[1:] # (num_frames, N)
seg_yvel = seg_df.pivot(index='frame_segment', columns='ID', values='y_vel').to_numpy()[1:]

## compute TDDC & max tau
## - TDDC : (N, N, num_tau, num_frames)
## - index (indices of max tau) : (N, N, num_frames)
_, TDDC, index = get_TDDC(seg_xvel, seg_yvel, SAMP_FREQ=config.SAMP_FREQ)
N = TDDC.shape[0]

if i_plotted == None: i_plotted = range(N)
if j_plotted == None: j_plotted = range(N)

for i in i_plotted:
    for j in j_plotted:
        if i != j:
            # print(i+1,j+1)
            title = f'Trial {trial} Segment {seg}'
            title += ': TDDC{}(t,$\\tau$)'.format(get_sub(str(i+1)+'-'+str(j+1)))
            for form in ['png','svg']:
                plot_TDDC_ij(TDDC[i,j,:,:], index[i,j,:],
                             SAMP_FREQ=config.SAMP_FREQ,
                             title=None,     # title,
                             saved=saved,
                             output_folder=f'../output/sayles/{form}/TDDC_heatmaps/',
                             output_file=f'TDDC_trial{int(trial)}_seg{seg}_i{i+1}_j{j+1}',
                             output_format=form)
                    
print('done')

# Plot networks

In [6]:
import numpy as np
import pandas as pd
import pickle
import config
from utils.get_network import plot_networks

# ===== CHANGE THIS ========================================
# the size of network time window (in s)
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
# ntwk_window_size = 1
# whether to save the image or show
saved = True
# ==========================================================

## init
ntwk_window_size_ms = int(ntwk_window_size * 1000)
unpruned_weights = pickle.load( open(f'../data/pickle/sayles_weights_unpruned_{ntwk_window_size_ms}ms.p', 'rb') )
pruned_weights = pickle.load( open(f'../data/pickle/sayles_weights_pruned_{ntwk_window_size_ms}ms.p', 'rb') )
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')

## constant values
SAMP_FREQ = config.SAMP_FREQ
num_frames_network = int(SAMP_FREQ * ntwk_window_size)

## plot networks
print('plotting networks (network window size:', ntwk_window_size, 's)')

for trial in unpruned_weights.keys():
    print('trial', int(trial), '-- segment', end=' ')
    trial_df = df[(df['trial']==trial)]

    for seg in unpruned_weights[trial].keys():
        print(seg, end=' ')
        # --- init ---
        num_networks, N, _ = unpruned_weights[trial][seg].shape # (num_networks, N, N)
        seg_df = trial_df[trial_df['segment']==seg]
        # get node positions for each network
        x = seg_df.pivot(index='frame_segment', columns='ID', 
                         values='x_transformed').to_numpy()     # values='x'
        y = seg_df.pivot(index='frame_segment', columns='ID', 
                         values='y_transformed').to_numpy()     # values='y'
        positions = np.stack((x[::num_frames_network, :], 
                              y[::num_frames_network, :]), 
                              axis=2)    # (num_networks, N, 2)
        # --- plot ---
        for form in ['png', 'svg']:
            ## unpruned
            plot_networks(unpruned_weights[trial][seg], positions,
                          order="ij",
                          saved=saved,
                          title=f'Unpruned networks in Trial{int(trial)} Seg{seg} (network window: {ntwk_window_size}s)',
                          output_folder=f'../output/sayles/{form}/networks/weights_{ntwk_window_size_ms}ms/',
                          output_file=f'unpruned_trial{int(trial)}_seg{seg}_network',
                          output_format=form)
            ## pruned 
            plot_networks(pruned_weights[trial][seg], positions,
                          order="ij",
                          saved=saved,
                          title=f'Pruned networks in Trial{int(trial)} Seg{seg} (network window: {ntwk_window_size}s)',
                          output_folder=f'../output/sayles/{form}/networks/weights_{ntwk_window_size_ms}ms/',
                          output_file=f'pruned_trial{int(trial)}_seg{seg}_network',
                          output_format=form)
    print(end='\n')

if (saved):
    print('saved!')

plotting networks (network window size: 1 s)
trial 1 -- segment 1 2 3 
trial 2 -- segment 1 2 3 4 5 
trial 3 -- segment 1 2 3 4 
trial 4 -- segment 1 2 3 4 5 
trial 5 -- segment 1 2 3 4 5 
trial 6 -- segment 1 2 3 4 5 
trial 7 -- segment 1 2 5 7 
trial 8 -- segment 1 2 3 4 5 
trial 9 -- segment 1 2 4 5 
trial 10 -- segment 1 3 4 
trial 11 -- segment 1 2 3 
trial 12 -- segment 1 2 4 
saved!


In [4]:
# Plot networks from specific trial, segment, and network IDs for publication

import numpy as np
import pandas as pd
import pickle
import config
from utils.get_network import plot_networks

# ===== CHANGE THIS ========================================
# the size of network time window (in s)
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
# ntwk_window_size = 1
# whether to save the image or show
saved = True
# specify networks to plot
trial = 10 #5
seg = 1 #5
start_ntwk_ID = 13 #11  # plot network IDs between start_network and end_network
end_ntwk_ID = 32 #15
# ==========================================================

## init
ntwk_window_size_ms = int(ntwk_window_size * 1000)
unpruned_weights = pickle.load( open('../data/pickle/sayles_weights_unpruned_' +
                                     str(ntwk_window_size_ms) + 'ms.p', 'rb') )
pruned_weights = pickle.load( open('../data/pickle/sayles_weights_pruned_' +
                                     str(ntwk_window_size_ms) + 'ms.p', 'rb') )
df = pd.read_csv('../data/csv/sayles_data_90pct.csv')
## constant values
SAMP_FREQ = config.SAMP_FREQ
num_frames_network = int(SAMP_FREQ * ntwk_window_size)

## plot networks

# --- init ---
print(f'plotting networks for Trial {trial} Segment {seg} Networks {start_ntwk_ID} - {end_ntwk_ID}',
      f'(network window size: {ntwk_window_size}s)')
trial_df = df[(df['trial']==trial)]
num_networks, N, _ = unpruned_weights[trial][seg].shape # (num_networks, N, N)
seg_df = trial_df[trial_df['segment']==seg]
# get node positions for each network
x = seg_df.pivot(index='frame_segment', columns='ID', 
                    values='x_transformed').to_numpy()     # values='x'
y = seg_df.pivot(index='frame_segment', columns='ID', 
                    values='y_transformed').to_numpy()     # values='y'
positions = np.stack((x[::num_frames_network, :], 
                        y[::num_frames_network, :]), 
                        axis=2)    # (num_networks, N, 2)

# --- plot ---
for form in ['png', 'svg']:
    ## unpruned
    plot_networks(unpruned_weights[trial][seg][start_ntwk_ID:end_ntwk_ID+1,:,:],
                  positions[start_ntwk_ID:end_ntwk_ID+1,:,:],
                  order="ij",
                  saved=saved,
                  subtitle='', # no title
                  output_folder=f"../output/sayles/{form}/networks/publication/",
                  output_file=f"unpruned_trial{trial}_seg{seg}_ntwk{start_ntwk_ID}to{end_ntwk_ID}_{ntwk_window_size_ms}ms",
                  output_format=form)
    ## pruned 
    plot_networks(pruned_weights[trial][seg][start_ntwk_ID:end_ntwk_ID+1,:,:], 
                  positions[start_ntwk_ID:end_ntwk_ID+1,:,:],
                  order="ij",
                  saved=saved,
                  subtitle='',   # no title
                  output_folder=f"../output/sayles/{form}/networks/publication/",
                  output_file=f"pruned_trial{trial}_seg{seg}_ntwk{start_ntwk_ID}to{end_ntwk_ID}_{ntwk_window_size_ms}ms",
                  output_format=form)

if (saved):
    print('saved!')

plotting networks for Trial 10 Segment 1 Networks 13 - 32 (network window size: 0.5s)
saved!


# Check how much the networks changed due to pruning
Compare the number of edges before vs. after pruning.

In [31]:
import numpy as np
import pickle
import config

# ===== CHANGE THIS ========================================
# the size of network time window (in s)
ntwk_window_size = config.NTWK_WINDOW_SIZE   # in s
# ==========================================================

## init
ntwk_window_size_ms = int(ntwk_window_size * 1000)
unpruned_weights = pickle.load( open(f'../data/pickle/sayles_weights_unpruned_{ntwk_window_size_ms}ms.p', 'rb') )
pruned_weights = pickle.load( open(f'../data/pickle/sayles_weights_pruned_{ntwk_window_size_ms}ms.p', 'rb') )

num_all_possible_edges = 0
num_TDDC_edges = 0       # the number of non-zero edges before pruning
num_pruned_edges = 0          # the number of edges that got pruned

# count the number of links if they were undirected
num_undirected_TDDC_edges = 0
num_undirected_pruned_edges = 0

for trial in unpruned_weights.keys():
    for seg in unpruned_weights[trial].keys():
        # unpruned_weights[trial][seg] : (num_networks, N, N)
        num_networks, N, _ = unpruned_weights[trial][seg].shape

        # check the number of TDDC edges (not zero or nan)
        unpruned_mask = (unpruned_weights[trial][seg] != 0)     # True if not 0, False if 0
        unpruned_mask[np.isnan(unpruned_weights[trial][seg])] = False   # False if NaN
        for n in range(num_networks):   # self-link should be False
            np.fill_diagonal(unpruned_mask[n], False)

        # check the number of pruned edges
        pruned_mask = (unpruned_weights[trial][seg] != pruned_weights[trial][seg])    # True if pruned; False if the same
        pruned_mask[np.isnan(unpruned_weights[trial][seg]) | np.isnan(pruned_weights[trial][seg])] = False  # False if NaN
        for n in range(num_networks):   # self-link should be False
            np.fill_diagonal(pruned_mask[n], False)

        # check the number of links if we 
        undirected_unpruned_mask = unpruned_mask | np.transpose(unpruned_mask, axes=(0, 2, 1))
        undirected_pruned_mask = pruned_mask | np.transpose(pruned_mask, axes=(0, 2, 1))

        num_all_possible_edges += num_networks * (N * N - N)      # exclude edge to the same node (i to i)
        num_TDDC_edges += np.count_nonzero(unpruned_mask)
        num_pruned_edges += np.count_nonzero(pruned_mask)

        num_undirected_TDDC_edges += int(np.count_nonzero(undirected_unpruned_mask)/2)
        num_undirected_pruned_edges += int(np.count_nonzero(undirected_pruned_mask)/2)

# print(round(num_pruned_edges/num_total_edges*100,2), "% of edges were pruned")
print(f"the number of all possible edges: {num_all_possible_edges}")
print(f"the number of non-zero edges before pruning: {num_TDDC_edges} ({round(num_TDDC_edges/num_all_possible_edges*100, 2)}% of all possible edges)")
print(f"the number of edges that got pruned: {num_pruned_edges} ({round(num_pruned_edges/num_all_possible_edges*100, 2)}% of all possible edges;",
      f"{round(num_pruned_edges/num_TDDC_edges*100, 2)}% of the edges initially found)")
print("--------")
print(f"This leaves {num_TDDC_edges-num_pruned_edges} edges ({round((num_TDDC_edges-num_pruned_edges)/num_all_possible_edges*100, 2)}% of all possible edges) in place")


num_all_possible_undirected_edges = int(num_all_possible_edges/2)
print()
print(f"the number of all possible edges if they were undirected: {num_all_possible_undirected_edges}")
print(f"the number of non-zero undirected edges before pruning: {num_undirected_TDDC_edges} ({round(num_undirected_TDDC_edges/num_all_possible_undirected_edges*100, 2)}% of all possible undirected edges)")
print(f"the number of non-zero undirected edges after pruning: {num_undirected_pruned_edges} ({round(num_undirected_pruned_edges/num_all_possible_undirected_edges*100, 2)}% of all possible undirected edges; {round(num_undirected_pruned_edges/num_undirected_TDDC_edges*100, 2)}% of the edges initially found)")


the number of all possible edges: 391160
the number of non-zero edges before pruning: 191020 (48.83% of all possible edges)
the number of edges that got pruned: 97609 (24.95% of all possible edges; 51.1% of the edges initially found)
--------
This leaves 93411 edges (23.88% of all possible edges) in place

the number of all possible edges if they were undirected: 195580
the number of non-zero undirected edges before pruning: 166544 (85.15% of all possible undirected edges)
the number of non-zero undirected edges after pruning: 84367 (43.14% of all possible undirected edges; 50.66% of the edges initially found)


# Plot distribution of weights

## 1. Save network weights for all relevant network time windows
Run the `Get network weights` section to get network weights for relevant time windows: `0.5s, 1s, 2s`.

## 2. Plot the distribution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

## ====== change ONLY HERE ============
ntwk_window_sizes = [0.5]
## ====================================


for ntwk_window_size in ntwk_window_sizes:

    ntwk_window_size_ms = int(ntwk_window_size * 1000)
    weights = {}

    for prune_type in ['unpruned', 'pruned']:
        weights[prune_type] = []
        data = pickle.load( open(f'../data/pickle/sayles_weights_{prune_type}_{ntwk_window_size_ms}ms.p', 'rb') )
        for trial in data.keys():
            for seg in data[trial].keys():
                num_networks, N, _ = data[trial][seg].shape
                mask = ~np.eye(N, dtype=bool)
                weights[prune_type].extend(data[trial][seg][:, mask].flatten())

    bins = np.arange(0, 1.2, 0.1)

    # Calculate frequency counts for unpruned and pruned data
    freq_unpruned, _ = np.histogram(weights['unpruned'], bins=bins)
    freq_pruned, _ = np.histogram(weights['pruned'], bins=bins)

    # Plot the histogram with grouped bars
    bar_width = 0.4
    opacity = 1

    fig, ax = plt.subplots(figsize=(11, 5))

    index = np.arange(len(bins) - 1)
    rects1 = ax.bar(index - bar_width/2, freq_unpruned, bar_width, alpha=opacity, color='tab:blue', label='Unpruned')
    rects2 = ax.bar(index + bar_width/2, freq_pruned, bar_width, alpha=opacity, color='tab:orange', label='Pruned')

    ax.set_xlabel('Weight')
    ax.set_ylabel('Frequency')
    if len(weights['unpruned']) != len(weights['pruned']):
        raise ValueError("something is wrong: # of edges")
    title = f"Distribution of {ntwk_window_size}s network weights (Unpruned vs Pruned)\ntotal: {len(weights['unpruned'])} edges"
    ax.set_title(title)
    ax.set_xticks(index)
    ax.set_xticklabels([f'{bins[i]:.1f}-{bins[i+1]:.1f}' for i in range(len(bins) - 1)])
    ax.legend()
    ax.grid(True, axis='y')

    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 2),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')

    autolabel(rects1)
    autolabel(rects2)

    plt.tight_layout()

    for form in ['png', 'svg']:
        new_folder = f'../output/sayles/{form}/weights_distrubution/'
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        plt.savefig(os.path.join(new_folder, f'weight_distribution_{ntwk_window_size_ms}ms.{form}'))
    # plt.show()
    plt.clf()
    plt.close()